# Vizualizace fraktálů

## Textový popis

Tento projekt se zaměřuje na vizualizaci fraktálů. Cílem je
implementovat algoritmy pro efektivní generování známých fraktálů, jako
jsou Mandelbrotova a Juliova množina (pro
$f\left(z\right)=z^{2}+c,c\in\mathbb{C}$), a vytvořit interaktivní
vizualizace (viz [Matplotlib
widgets](https://matplotlib.org/stable/gallery/widgets/index.html))
těchto fraktálů pomocí knihovny Matplotlib. Možností je také využít knihovnu [Pygame](https://www.pygame.org/news) pro vytvoření interaktivních vizualizací.

Výstupem projektu budou interaktivní vizualizace fraktálů, které
umožňují uživateli prozkoumávat různé části fraktálu a přizpůsobovat
parametry pro generování fraktálů ($c\in\mathbb{C}$ pro Juliovu
množinu).

## Funkcionality

- Implementovat algoritmus pro efektivní generování Mandelbrotovy
  množiny pomocí knihoven NumPy a Numba
- Implementovat algoritmus pro efektivní generování Juliovy množiny
  (pro $f\left(z\right)=z^{2}+c,c\in\mathbb{C}$) pomocí knihoven NumPy a Numba
- Vytvořit funkci pro vizualizaci fraktálů pomocí knihovny Matplotlib,
  která zobrazuje fraktály pomocí barevného mapování podle iterací
  potřebných k dosažení určitého prahu
- Implementovat interaktivní prvky vizualizace, které umožňují
  uživateli (v případě implementace skrze Pygame lze nahradit ovladáním klávesnicí a myší):
  - přiblížit nebo oddálit fraktál
  - měnit barevné schéma vykreslení počtu iterací do divergence
  - přizpůsobovat parametry pro generování fraktálů (např. počet
    iterací, $c$)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
from ipywidgets import interact, IntSlider, ToggleButtons, FloatSlider

## Interaktivni implementace Mandelbrotovy mnoziny

In [2]:
@numba.njit(parallel=True)
def mandelbrot_set(x_min=-2.0, x_max=1.0, y_min=-1.5, y_max=1.5, n=1000, k=100):
    re = np.linspace(x_min, x_max, n)
    im = np.linspace(y_min, y_max, n)
    
    divergence_matrix = np.full((n, n), k, dtype=np.int32)
    
    # Prochazeni vsech bodu v rovine
    for ix in numba.prange(n):
        for iy in range(n):
            c = re[ix] + 1j * im[iy]
            z = 0.0 + 0.0j
            
            # Iterace pro vypocet Mandelbrotovy posloupnosti
            for i in range(k):
                z = z * z + c
                
                # Kontrola divergence, pokud |z| > 2 (bez pouziti numpy, nebot to zpomaluje celou funcki)
                if (z.real * z.real + z.imag * z.imag) > 4.0:
                    divergence_matrix[ix, iy] = i
                    break
                    
    return divergence_matrix


In [3]:
n = 500
k = 100

In [37]:
%timeit mandelbrot_set(n=n, k=k)

3.21 ms ± 131 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def plot_mandelbrot(n=500, k=100, cmap='hot'):
    data = mandelbrot_set(n=n, k=k)
    plt.figure(figsize=(9, 8))
    plt.imshow(data.T, cmap=cmap, origin='lower', extent=[-2.0, 1.0, -1.5, 1.5])
    plt.colorbar(label='Počet iterací do divergence')
    plt.title(f'Mandelbrotova množina (k={k}, n={n})')
    plt.show()

# Interaktivní posuvníky
interact(plot_mandelbrot,
         n=IntSlider(min=50, max=2500, step=50, value=500, description='Rozlišení'),
         k=IntSlider(min=5, max=500, step=5, value=100, description='Iterace'),
         cmap=ToggleButtons(options=['hot', 'viridis', 'inferno', 'gray', 'turbo'], value='hot', description='Barevna mapa'));

interactive(children=(IntSlider(value=500, description='Rozlišení', max=2500, min=50, step=50), IntSlider(valu…

## Interaktivni implementace Juliovy mnoziny